In [1]:
from matplotlib import pyplot as plt
from scipy.special import softmax
from itertools import product
import numpy as np
from utils import *
from model import *
import time
import seaborn as sns
from glob import glob as glob
sns.set_context('talk')

%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
gsname = 'gs0125'

### load data

In [ ]:
# ## run make_df-1102.py to make dfs
datadf = datadf1 = pd.read_csv('data/%s-datadf-1.csv'%gsname)
datadf = datadf1 = pd.read_csv('data/%s-datadf-0.csv'%gsname)


In [ ]:
gsdf = pd.read_csv('data/%s-summdf.csv'%gsname).astype(float)
gsdf

In [ ]:
## human data
hdf = pd.read_csv('../human_data.csv')
humanD = {}
for c in ['blocked','interleaved','early','middle','late']:
  humanD[c[0]] = hdf.loc[:,('%s mean')%c].values.T[-40:].mean()
## difference to human
for c in ['b','i','e','m','l']:
  gsdf['%s-test_diff'%c] = np.abs(gsdf.loc[:,'testacc-%s'%c] - humanD[c])

In [ ]:
# datadf.columns
# paramL = ['concentration', 'stickiness_wi', 'stickiness_bt',
#        'sparsity', 'pvar', 'lrate', 'lratep', 'decay_rate']

In [ ]:
gsdf

In [ ]:
gsdf.loc[:,'mse-bi'] = gsdf.loc[:,'mse-i']+gsdf.loc[:,'mse-b']

### compute df metrics, form summary df

In [ ]:
# subset of dataframe containing relevant params
Btest = gsdf.loc[:,'testacc-b'] > 0.95
Itest1 = gsdf.loc[:,'testacc-i'] > 0.6
Itest2 = gsdf.loc[:,'testacc-i'] < 0.65
Etest = gsdf.loc[:,'testacc-e'] > 0.8
Mtest = gsdf.loc[:,'testacc-m'] < 0.9
Mtest2 = gsdf.loc[:,'testacc-m'] < 0.85
Ltest = gsdf.loc[:,'testacc-l'] < 0.9
Iblock2 = gsdf.loc[:,'acc2-i'] > 0.51
skipt1 = gsdf.loc[:,'skipt1'] == 0
MItest = gsdf.loc[:,'testacc-m'] - gsdf.loc[:,'testacc-i'] > 0.1
LItest = gsdf.loc[:,'testacc-l'] - gsdf.loc[:,'testacc-i'] > 0.1

C = Btest&Mtest&Ltest
sgsdf = gsdf[C]
sgsdf = sgsdf.sort_values('testacc-l',ascending=False)
sgsdf

### plt single condition

In [ ]:
def get_cond_df(row):
  ## form query string for above row
  query_str = " and ".join(
    ["%s == %f"%(k,v) for k,v in dict(row.astype(float)[1:10].round(4)).items()]
  )
  print('query',query_str)
  ## df of single parameter condition`
  cond_df = datadf1.round(4).query(query_str)
  if not len(cond_df):
    cond_df = datadf2.round(4).query(query_str)
  paramstr = "__".join(["%s-%s"%(k,v) for k,v in cond_df.iloc[0][1:12].items()])
  print('found',paramstr)
  return cond_df


In [ ]:
def plt_cond_df(cond_df):
  f,axar = plt.subplots(1,1,figsize=(20,8))
  ax=axar
  paramstr = "__".join(["%s-%s"%(k,v) for k,v in cond_df.iloc[0][1:12].items()])
  for c,cdf in cond_df.groupby('cond'):
    try:
#       if c not in ['blocked','interleaved']:
#         print('FIGURES WITH BI ONLY')
#         continue
      ax.plot(np.arange(200),cdf.acc,label=c)
    except:
      print('err',paramstr,c)
      continue
  ax.set_ylim(0,1)
  plt.legend()
  plt.axhline(0.5,c='k',ls='--')
  
  plt.title("%s\n%s"%(paramstr[:94],paramstr[94:]))
  plt.savefig('figures/analysis_gs/acc-%s.png'%paramstr)


In [ ]:
## select single param for plotting full acc trace
row = sgsdf.sort_values('testacc-l',ascending=False).iloc[0]
# print(row.astype(float))
cond_df = get_cond_df(row)

In [ ]:
## TOP PARAM BY MSE
row = gsdf.sort_values('mse-bi').iloc[0]
print(row.astype(float))
cond_df = get_cond_df(row)
plt_cond_df(cond_df)
# plt.savefig('figures/gs0125-topMSE')

### plot full subtable

In [ ]:
for idx,row in sgsdf.iterrows():
  print()
  cond_df = get_cond_df(row)
  plt_cond_df(cond_df)
  plt.close('all')